# 23 Day 05

https://adventofcode.com/2022/day/5

## --- Day 5: Supply Stacks ---

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a **giant cargo crane** capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which **crate** will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates **and** the rearrangement procedure (your puzzle input). For example:

        [D]    
    [N] [C]    
    [Z] [M] [P]
     1   2   3 

    move 1 from 2 to 1
    move 3 from 1 to 3
    move 2 from 2 to 1
    move 1 from 1 to 2

In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

    [D]        
    [N] [C]    
    [Z] [M] [P]
     1   2   3 

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved **one at a time**, so the first crate to be moved (D) ends up below the second and third crates:

            [Z]
            [N]
        [C] [D]
        [M] [P]
     1   2   3

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved **one at a time**, crate C ends up below crate M:

            [Z]
            [N]
    [M]     [D]
    [C]     [P]
     1   2   3

Finally, one crate is moved from stack 1 to stack 2:

            [Z]
            [N]
            [D]
    [C] [M] [P]
     1   2   3

The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?


In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2022, day=5)

In [7]:
import re
from dataclasses import dataclass

@dataclass
class Procedure:
    amount: int
    source: int
    dest: int


class CargoShip:
    def __init__(self, raw_arrangement, raw_procedures):
        self.stack_labels, self.stacks = CargoShip.parse_arrangement(raw_arrangement)
        self.procedures = CargoShip.parse_procedures(raw_procedures)

    def __str__(self):
        tallest_stack = max(self.stacks, key=len)
        CONTAINER_FORMAT = ' [ {0} ] '
        LABEL_FORMAT = CONTAINER_FORMAT.replace('[', ' ').replace(']', ' ')

        output = ''
        for i in reversed(range(len(tallest_stack))):
            for stack in self.stacks:
                output += CONTAINER_FORMAT.format(stack[i] if i < len(stack) else ' ')
            output += '\n'
        for label in self.stack_labels:
            output += LABEL_FORMAT.format(label)
        output += '\n\n'
        output += '\n'.join([f'move {procedure.amount} from {procedure.source} to {procedure.dest}' for procedure in self.procedures])
        return output

    def __iter__(self):
        return self

    def __next__(self):
        if len(self.procedures) == 0:
            raise StopIteration
        p = self.procedures.pop(0)
        return self.stacks[p.dest - 1].extend([self.stacks[p.source - 1].pop() for _ in range(p.amount)])

    def get_top(self) -> str:
        return ''.join([stack[-1] for stack in self.stacks])

    @classmethod
    def parse_arrangement(cls, arrangement) -> [[str], [str]]:
        rows = iter(reversed(arrangement.split('\n')))
        stack_labels = next(rows).split()
        stacks = [[] for _ in range(len(stack_labels))]
        for row in rows:
            containers = re.findall(r'\[(.)\]|    ', row)
            for indx, c in enumerate(containers):
                if c:
                    stacks[indx].append(c)      

        return stack_labels, stacks

    @classmethod
    def parse_procedures(cls, procedures) -> list[Procedure]:
        return [Procedure(int(m.group('amount')), int(m.group('source')), int(m.group('dest'))) for m in [re.match(r'move (?P<amount>\d+) from (?P<source>\d+) to (?P<dest>\d+)', procedure) for procedure in procedures.split('\n') if procedure]]

 [   ]  [ D ]  [   ] 
 [ N ]  [ C ]  [   ] 
 [ Z ]  [ M ]  [ P ] 
   1      2      3   

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
CMZ


In [8]:
data = """    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""

arrangement, procedures = data.split('\n\n')
cs = CargoShip(arrangement, procedures)
print(cs)
for _ in cs:
    pass
print(cs.get_top())

 [   ]  [ D ]  [   ] 
 [ N ]  [ C ]  [   ] 
 [ Z ]  [ M ]  [ P ] 
   1      2      3   

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
CMZ


In [11]:
arrangement, procedures = puzzle.input_data.split('\n\n')
cs = CargoShip(arrangement, procedures)
print(cs)
for _ in cs:
    pass
print(f'\nPart A Solution: {cs.get_top()}')

 [   ]  [   ]  [   ]  [ Q ]  [   ]  [ G ]  [   ]  [ M ]  [   ] 
 [   ]  [   ]  [   ]  [ B ]  [ S ]  [ V ]  [   ]  [ P ]  [ R ] 
 [   ]  [ T ]  [   ]  [ C ]  [ F ]  [ L ]  [   ]  [ V ]  [ N ] 
 [ Q ]  [ P ]  [   ]  [ H ]  [ N ]  [ S ]  [   ]  [ W ]  [ C ] 
 [ F ]  [ G ]  [ B ]  [ J ]  [ B ]  [ N ]  [   ]  [ Z ]  [ L ] 
 [ L ]  [ Q ]  [ Q ]  [ Z ]  [ M ]  [ Q ]  [ F ]  [ G ]  [ D ] 
 [ S ]  [ Z ]  [ M ]  [ G ]  [ H ]  [ C ]  [ C ]  [ H ]  [ Z ] 
 [ R ]  [ N ]  [ S ]  [ T ]  [ P ]  [ P ]  [ W ]  [ Q ]  [ G ] 
   1      2      3      4      5      6      7      8      9   

move 1 from 2 to 6
move 3 from 7 to 9
move 7 from 9 to 4
move 2 from 5 to 3
move 3 from 2 to 8
move 14 from 4 to 5
move 1 from 2 to 1
move 1 from 2 to 3
move 3 from 6 to 8
move 3 from 6 to 9
move 1 from 4 to 6
move 5 from 9 to 8
move 9 from 8 to 9
move 3 from 3 to 8
move 8 from 9 to 4
move 2 from 1 to 7
move 4 from 1 to 5
move 2 from 7 to 1
move 1 from 9 to 6
move 7 from 4 to 7
move 1 from 8 to 4
move 1 from 9 to 8
move